# IB-Servidor de Datos de Ticks

Autor: David U. Ordiz.

**Descripción del Notebook:**

En este notebook vamos a utilizar la librería ib_insync para solicitar datos de mercado a InteractiveBrokers y almacenarlos en una base de datos. La librería *ib_insync* utiliza el protocolo WebSockets para obtener datos de mercado usando un enfoque multihilo.

En concreto, a en este Notebook veremos como obtener datos de tick del EURUSD (Forex), y como almacenar dichos datos en una base de datos SQL. Dicha base de datos se actualizará cada vez que recibamos un nuevo tick. En posteriores Notebooks veremos cómo acceder a estos datos para procesarlos, generar señales, y enrutar ordernes a IB en tiempo real. 

**Pasos realizados en este Notebook:**

1. Importamos las librerías necesarias.
2. Establecemos la conexión a la API de InteractiveBrokers.
3. Solicitamos datos de ticks del EURUSD (Forex).
4. Creamos el motor para nuestra base de datos SQL usando la librería sqlalchemy.
5. Por último, almacenamos en la base de datos cada nuevo tick recibido del EURUSD.

####  1. Importamos las librerías necesarias

In [1]:
import sqlalchemy # Librería que permite el uso y manejo de bases de datos relacionales (SQL)
import pandas as pd
import numpy as np
from IPython.display import clear_output # Permite limpiar el resultado de una celda.
from datetime import datetime, timedelta
import time
from ib_insync import *
# Inicia un loop para mantener el notebook continuamente actualizado. 
util.startLoop() 

####  2. Establecemos la conexión con InteractiveBrokers (IB)

In [2]:
# Crea un objeto IB
ib = IB()
# se conecta a IB
ib.connect('127.0.0.1', 7497, clientId = 6)

<IB connected to 127.0.0.1:7497 clientId=6>

#### 3. Solicitamos datos de ticks del EURUSD (Forex).

In [3]:
# La función reqMktData permite solicitar la transmisión de ticks.
ib.reqMktData(Forex('EURUSD'), '', False, False)
print("Datos de mercado solicitados correctamente.");

Datos de mercado solicitados correctamente.


#### 4. Creamos el motor para nuestra base de datos usando la librería sqlalchemy.

In [4]:
# La función create_engine() crea un motor para nuestra base de datos. 
# Dicho motor nos permite conectarnos y manipular la base de datos. 
# Más información: https://docs.sqlalchemy.org/en/20/core/engines.html
engine = sqlalchemy.create_engine('sqlite:///BaseDeDatosIB.db')
print("El motor de la base de datos ha sido creado correctamente.");

El motor de la base de datos ha sido creado correctamente.


#### 5. Almacenamos en la base de datos cada nuevo tick recibido del EURUSD.

In [5]:
# Esta variable contendrá el útlimo tick almacenado en la base de datos
PrevLastPrice = 0 
# Esta variable almacenará el tiempo del último tick añadido a la base de datos.
# Iniciamos esta variable con el tiempo actual.
PreviousTime = datetime.now()

# La función onPendingTickers nos permite almacenar en la base de datos
# cada nuevo tick recibido del EURUSD.
# Esta función se ejecuta cada vez que hemos recibido un nuevo tick.
def onPendingTickers(tickers):   

    # Este bucle for sirve para revisar cada símbolo que ha recibido datos.
    # Si solo hemos solicitado datos de un símbolo, entonces, solo revisará uno.
    # En este caso el EURUSD.
    for symbol in tickers:

        # Variables globales
        global PrevLastPrice
        global PreviousTime

        # Almacena el precio de mercado recibido y el tiempo actual.   
        LastPrice = symbol.marketPrice() 
        Time = datetime.now()

        # Limpia el resultado de la celda. Esto nos sirve para limpiar el último valor 
        # impreso en la celda
        clear_output(wait=True)

        # Antes de almacenar un nuevo tick en la base de datos comprobaremos 3 cosas:
        # 1) El tiempo (Time) del tick actual debe ser mayor que el último tick almacenado
        # en la base de datos + varios milisegundos (Por defecto 200)
        # 2) El último tick recibido (LastPrice) debe ser distinto al último tick almacenado
        # en la base de datos.
        # 3) El valor del último tick recibido (LastPrice) no debe ser nulo (NaN).
        # Si estas tres condiciones son ciertas entonces... 
        if (Time > (PreviousTime + timedelta(milliseconds = 200))
            and LastPrice != PrevLastPrice
            and np.isnan(LastPrice) == False):
            
            # Crea un nuevo dataframe con los valores de las variables
            # actualizadas (Time y LastPrice)
            df = pd.DataFrame({'Time': [Time], 'LastPrice': [LastPrice]})
            df.set_index('Time', inplace=True) # Configura el índice

            # Adjunta el DataFrame a la tabla EURUSD de la base de datos.
            df.to_sql('EURUSD', engine, if_exists = 'append', index='Time')   

            # Imprime los valores adjuntados a la base de datos. 
            print("Último tick del EURUSD añadido a la base de datos: " 
                  "|", Time, "|", LastPrice)

            # Muestra el precio y la fecha y hora del úlitmo tick alamcenado.
            PrevLastPrice = LastPrice  
            PreviousTime = Time
        # Si alguna de las tres condiciones no es cierta entonces...    
        else: 
            print("Último tick del EURUSD añadido a la base de datos: "
                  "|", PreviousTime, "|", PrevLastPrice)

# Fin del método onPendingTickers().

# La función pendingTickersEvent() emite el grupo de símbolos que han obtenido 
# nuevos ticks en la última actualización.
# Aquí solo solicitamos datos de un símbolo (EURUSD). Por tanto, cada vez que
# recibamos un tick de este símbolo, se ejecutará la función oppendingTickers()
ib.pendingTickersEvent += onPendingTickers
# La función run() mantiene el programa corriendo.
ib.run()

Último tick del EURUSD añadido a la base de datos: | 2023-05-31 18:17:34.955858 | 1.0642
